In [0]:
# １．実行環境準備

# 必要なライブラリのインポート
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import Adam
import tensorflow as tf
import random as rn
import numpy as np
import os

In [0]:
# 実行の設定
INPUT_SHAPE = (1, 28, 28)
CLASSES_NUM = 10
EPOCH_NUM = 3  # 50
BATCH_SIZE = 128
K.set_image_dim_ordering('th')
SEED = 33

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(SEED)
rn.seed(SEED)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

tf.set_random_seed(SEED)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
# ２．学習データ準備

# データの読み込み
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
# データ量の確認
(x_train.shape, x_test.shape)

In [0]:
# データの中身の確認（イメージ）
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)
for i in range(100):
    image = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    image.imshow(x_train[i].reshape((28, 28)), cmap='gray')

In [0]:
# データの中身の確認（イメージに対する正解値）
for i in range(10):
    print(y_train[i*10:i*10+10])

In [0]:
# 一つだけ取り出して確認（格納バイナリ値）
import pandas as pd
num_image=x_train[0].reshape(28,28)
pd.options.display.max_columns=28
num_binary=pd.DataFrame(num_image)
num_binary

In [0]:
# 一つだけ取り出して確認（イメージ）
plt.imshow(x_train[0].reshape(28, 28))

In [0]:
# 正解データの確認（正解値）
print(y_train[0])

In [0]:
# データの正規化
x_train = x_train[:, np.newaxis, :, :]
x_test = x_test[:, np.newaxis, :, :]

_y_train = y_train
_y_test = y_test


y_train = np_utils.to_categorical(
    y=y_train, num_classes=CLASSES_NUM)
y_test = np_utils.to_categorical(
    y=y_test, num_classes=CLASSES_NUM)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# ３．ディープランニング実行

# 【①CNNモデル構築】
model = Sequential()

model.add(Conv2D(filters=20, kernel_size=5, padding='same',
           input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=50, kernel_size=5, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))

model.add(Dense(units=CLASSES_NUM))
model.add(Activation('softmax'))

In [0]:
# 【②学習】
model.compile(
    loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
history = model.fit(
    x=x_train, y=y_train, batch_size=BATCH_SIZE,
    epochs=EPOCH_NUM, verbose=2, validation_split=0.2)


In [0]:
# 予測用のデータの確認
#plt.imshow(x_test[0].reshape(28, 28))

s = 1210
e = s + 30

fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)
c = 0
for i in range(s, e):
    image = fig.add_subplot(10, 10, c + 1, xticks=[], yticks=[])
    image.imshow(x_test[i].reshape((28, 28)), cmap='gray')
    c = c + 1

In [0]:
# 【③予測】
predict_classes = model.predict_classes(x_test[s:e,])

predict_classes.reshape(3, 10)

In [0]:
# ４．予測結果の検証
true_classes = np.argmax(y_test[s:e],1)
true_classes.reshape(3, 10)

In [0]:
# Confusion matrix(混同行列)の確認 
from sklearn.metrics import confusion_matrix
print(confusion_matrix(true_classes, predict_classes))

In [0]:
# 精度の確認
model.evaluate(x_test, y_test)

In [0]:
####################################################
# 上級者用 (追加部分)                              #
####################################################

In [0]:
from PIL import Image
from google.colab import files

files.upload()
img=Image.open("infile.png").convert('L')
indata = img_array=np.array(img.resize((28, 28)))

In [0]:
# データの中身の確認
plt.imshow(indata)

In [0]:
# データの正規化
indata = img_array[ np.newaxis, np.newaxis, :, :]
indata = indata.astype('float32')
indata /= 255

In [0]:
# 【③予測】
predict_classes = model.predict_classes(indata)

print(predict_classes)